In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import pandas_gbq
from google.cloud import bigquery
%load_ext google.cloud.bigquery


In [ ]:
progress_bar_type = 'tqdm_notebook'
project_id = 'proof-time-prediction'

In [ ]:
import os
data_dir = os.path.realpath(os.path.join(
    os.getcwd(), "..", "data", "chains")) + os.sep
print(data_dir)

In [ ]:
chain_tables = [
    {'chain_table': 'raw_arbitrum', 'effective_gas_price': 'effective_gas_price'},
    {'chain_table': 'raw_base', 'effective_gas_price': 'effective_gas_price'},
    {'chain_table': 'raw_bsc', 'effective_gas_price': ''},
    {'chain_table': 'raw_ethereum',
        'effective_gas_price': 'receipt_effective_gas_price'},
    {'chain_table': 'raw_optimism', 'effective_gas_price': ''},
    {'chain_table': 'raw_zksync', 'effective_gas_price': ''},
    {'chain_table': 'raw_avalanche', 'effective_gas_price': ''},
    {'chain_table': 'raw_fantom', 'effective_gas_price': ''},
]

In [ ]:
for chain in chain_tables:
    os.makedirs(data_dir + chain['chain_table'][4:], exist_ok=True)

In [ ]:
sql = """
WITH view_incriptions AS (
  SELECT
    txs.hash as tx_hash,
    from_address as tx_from_address,
    to_address as tx_to_address,
    block_timestamp,
    block_number,
    gas,
    gas_price,
    receipt_gas_used,
    value,
    transaction_index,
    receipt_status,
    input as tx_input,
    {effective_gas_price}
  FROM
    nansen-query.{chain_table}.transactions as txs
  WHERE input LIKE '0x646174613a%' AND block_timestamp >= '{start_date}' AND block_timestamp < '{end_date}'
)
SELECT
  *
FROM
  view_incriptions
ORDER BY block_number DESC
"""

In [ ]:
start_date, end_date = '2023-01-01', 'today'
dates = list(map(lambda x: x.date(), pd.date_range(
    start=start_date, end=end_date, freq='MS')))

for chain in tqdm(chain_tables, desc='Gathering data'):
    chain_name = chain['chain_table'][4:]
    print(f'Gathering data for {chain_name}')
    os.makedirs(data_dir+chain_name, exist_ok=True)
    for end_date in dates[1:]:
        query = sql.format(
            chain_table=chain['chain_table'],
            effective_gas_price=chain['effective_gas_price'],
            start_date=str(start_date),
            end_date=str(end_date))

        df = pandas_gbq.read_gbq(
            query, project_id=project_id, progress_bar_type=progress_bar_type)

        filename = chain_name + \
            "/{}_{}_{}.csv.gz".format(chain['chain_table'],
                                      str(start_date), str(end_date))
        df.to_csv(data_dir+filename, index=False, compression='gzip')

        print(f"{chain['chain_table']} {str(start_date)} {str(end_date)} done")
        start_date = end_date